Imports

In [2]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
import time
import os
import copy
from PIL import Image
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time

# Set GPU Settings
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['TORCH_USE_CUDA_DSA'] = '1'

#TURN OFF WHEN ACTUALLY TESTING CODE
import warnings
warnings.filterwarnings("ignore")

Data Setup

In [3]:
# Define CarDataSet Class
class CarDataSet():

    # Define The Initialization
    def __init__(self, csv_file, root_dir, transform=None, target_transform=None):
        self.cars = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.target_transform = target_transform
        self.resize = transforms.Resize((150,150))  # Resize images to a uniform size

    # Define The Length Function
    def __len__(self):
        return len(self.cars)

    # Define The Get Item Function
    def __getitem__(self, idx):

        # Pull The Image And Check Settings
        img_name = os.path.join(self.root_dir, self.cars.iloc[idx, 0])

        image = Image.open(img_name)

        if image.mode != 'RGB':
          image = image.convert('RGB')

        # Pull The Label, -1 To Normalize To 0
        label = (self.cars.iloc[idx, 5]) - 1

        if self.transform:
            image = self.transform(image)

        # Define The Dictionary
        sample = {'image': image, 'cars': label}

        # Return
        return sample

In [4]:
# Define Transform
transform = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])
    
# Load The Data
train_data = CarDataSet(csv_file='../SynchronizationProject/stanford_cars_eec174/train_make.csv',
                                        root_dir='../SynchronizationProject/stanford_cars_eec174/images/train', transform=transform)
test_data = CarDataSet(csv_file='../SynchronizationProject/stanford_cars_eec174/val_make.csv',
                                        root_dir='../SynchronizationProject/stanford_cars_eec174/images/val', transform=transform)

In [5]:
def show_imgs(dataloader):
    # Load Names From The File
    with open('../SynchronizationProject/stanford_cars_eec174/names_make.txt', 'r') as file:
        names = file.read().splitlines()

    # Retrieve The Images And Labels
    dataiter = iter(dataloader)
    samples = next(dataiter)

    images_show_img, labels_show_img = samples['image'], samples['cars']

    # Grab Ten Random Indices
    shuffled_indices = np.random.permutation(len(images_show_img))
    indices = shuffled_indices[:10]

    # Create Subplots
    figure, axes = plt.subplots(1, 10, figsize=(20, 10))

    for i, ax in zip(indices, axes):

        # Rearrange Dimensions For Display
        cur_image = images_show_img[i].permute(1, 2, 0)

        # Convert The Label To An Integer
        label_index = int(labels_show_img[i].item())

        # Assign The Name Corresponding To The Label Index
        cur_label = names[label_index]

        # Display The Image
        ax.imshow(cur_image)

        # Display The Label As Title
        ax.set_title(cur_label)

        # Turn Off The Axis
        ax.axis('off')

    plt.show()

# Test Accuracy
def test_accuracy(model, test_loader_internal, passed_device, loss_fn):

    # Set Parameters
    model.to(passed_device)
    correct = 0
    total = 0
    run = 0
    val_loss = 0

    # Run Tests
    with torch.no_grad():
        for test_data_internal in test_loader_internal:
            images_test_acc, labels_test_acc = test_data_internal['image'].cuda(), test_data_internal['cars'].cuda()
            outputs_test_acc = model(images_test_acc)
            _, predicted_test_acc = torch.max(outputs_test_acc.data, 1)
            val_loss += (loss_fn(outputs_test_acc, labels_test_acc)).item()
            total += labels_test_acc.size(0)
            correct += (predicted_test_acc == labels_test_acc).sum().item()
            run += 1

    # Return
    return (100 * correct / total), val_loss/run


# Plotting Function
def general_plot(data_dict1: dict, data_dict2: dict, param: str, label1: str, label2: str, ylabel: str, reduce_noise: bool):
    param_list1 = data_dict1[param]
    param_list2 = data_dict2[param]

    if reduce_noise:
        param_list1.pop(0)
        param_list2.pop(0)

    # Prepare Plot
    xs = [x for x in range(min(len(param_list1), len(param_list2)))]
    plt.figure(figsize=(20, 10))
    plt.plot(xs, param_list1[:len(xs)], label=label1)
    plt.plot(xs, param_list2[:len(xs)], label=label2)
    plt.xlabel('Iterations (in batches)')
    plt.ylabel(ylabel)
    plt.title(f'Training and Validation {ylabel} Curve')
    plt.legend()
    plt.show()
    
    
# Plot Learning Curves
def plot_learning_curve(train_history_plt_curve: dict, val_history_plt_curve: dict, reduce_noise=True):

    # Plot The Info
    general_plot(train_history_plt_curve, val_history_plt_curve, 'loss', 'Training Loss', 'Validation Loss', 'Loss', reduce_noise)
    general_plot(train_history_plt_curve, val_history_plt_curve, 'accuracy', 'Training Accuracy', 'Validation Accuracy', 'Accuracy', reduce_noise)
    
# Define and use confusion matrix
def plot_confusion_matrix(y_true, y_pred, class_names_confusion):

    # Prepare Plot And Matrix
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(20, 20))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion Matrix Frozen')
    plt.colorbar()
    tick_marks = np.arange(len(class_names_confusion))
    plt.xticks(tick_marks, class_names_confusion, rotation=45)
    plt.yticks(tick_marks, class_names_confusion)

    fmt = 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], fmt),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")

    # Show Everything
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.tight_layout()
    plt.show()


# Define Train
def train(model, loss_fn, optimizer_train, train_loader_train, val_loader, num_epochs_train, device_train):

    # Set Parameters
    model.train()
    BLUE = '\033[94m'
    RESET = '\033[0m'

    train_history_train = {'loss': [], 'accuracy': []}
    val_history_train = {'loss': [], 'accuracy': []}

    total_start_time = time.time()

    # Iterate Through All Epochs
    for epoch in range(num_epochs_train):

        # For Loop Parameters
        start_time = time.time()
        total = 0
        correct = 0
        train_loss = 0
        train_correct = 0
        train_total = 0

        # Iterate Through The Training Dataset
        for i, data_train in enumerate(train_loader_train, 0):
            
            # Flatten Images And Load Data
            images_train, labels_train = data_train['image'].cuda(), data_train['cars'].cuda()

            # Zero Collected Gradients At Each Step (basically cleaning)
            optimizer_train.zero_grad()

            # Forward Propagate
            outputs_train = model(images_train)

            # Calculate Loss
            loss = loss_function(outputs_train, labels_train)

            # Back Propagate
            loss.backward()

            # Update Weigh Gradsts
            optimizer_train.step()

            # Calculate Accuracy
            _, predicted_train = torch.max(outputs_train.data, 1)
            total += labels_train.size(0)
            correct += (predicted_train == labels_train).sum().item()

            train_loss += loss.item()
            train_correct += 100 * correct / total
            train_total += 1
            
        # Evaluate Model
        model.eval()
        with torch.no_grad():

            val_acc, val_loss = test_accuracy(model, val_loader, device_train, loss_fn)

            val_history_train['loss'].append(val_loss)
            val_history_train['accuracy'].append(val_acc)

            train_history_train['loss'].append(train_loss / train_total)
            train_history_train['accuracy'].append(train_correct / train_total)

            train_loss = 0
            train_correct = 0
            train_total = 0
        model.train()


        # At End Of Each Epoch Print Duration And Accuracy
        print(f'{BLUE}Epoch [{epoch+1}/{num_epochs_train}]:'f' Duration: {round(time.time() - start_time, 2)}s |'f' Train Acc: {round(train_history_train["accuracy"][-1], 2)} |'f' Train Loss: {round(train_history_train["loss"][-1], 5)}'f' Val Acc: {round(val_history_train["accuracy"][-1], 2)} |'f' Val Loss: {round(val_history_train["loss"][-1], 5)}{RESET}')
        print('------------------------------------------------------------------------------------------------------------')

        # Save Checkpoint
        #PATH = '/content/drive/My Drive/WEIGHTSAVES/run' + str('78.8save') + 'epoch' + str(epoch+1) + 'ta' + str(train_history_train["accuracy"][-1])[0:6] + 'tl' + str(train_history_train["loss"][-1])[0:6] + 'va' + str(val_history_train["accuracy"][-1])[0:6] + 'vl' + str(val_history_train["loss"][-1])[0:6] + '.pth'
        #print(PATH)
        #torch.save(net.state_dict(), PATH)

    # Print Final Statistics
    #print(f'{BLUE}Total Duration:{round(time.time() - total_start_time, 2)}s |',f'Final Train Acc: {round(train_history_train["accuracy"][-1], 2)} |',f'Final Train Loss: {round(train_history_train["loss"][-1], 5)} |',f'Final Val Acc: {round(val_history_train["accuracy"][-1], 2)} |',f'Final Val Loss: {round(val_history_train["loss"][-1], 5)}{RESET}')

    # Return
    return train_history_train, val_history_train

Unchanging Definitions

In [5]:
#[ batchsize(32) * num, learningrate(0.0001) * num, GPUcount num, pretrained (True=yes, False=no),  

operations_dictionary = {
    "0": [1, 1, 1, False],
    "1": [1, 1, 3, False],
    "2": [2, 1, 1, False],
    "3": [2, 1, 5, False]
                         }
# Dataset Size
DATASET_SIZE = 8192

# Define Parameters
input_size = (768 * 1024)
num_classes = 49

InfoSave = pd.DataFrame(columns = ["Run", "BatchSize", "LearningRate", "TrainAccuracy", "TrainLoss", "TrainHistory", "ValAccuracy", "ValLoss", "ValHistory", "GPUCount", "GPUNumber", "PreTrain", "SavePathInput", "SavePathOutput"])

Model Setup and Training

In [6]:
# Run Each Set of Models 
for run_number_key in operations_dictionary:
    print(run_number_key)
    # Batch Size
    batch_size = 32 * (operations_dictionary[run_number_key])[0]
    
    # Import To Dataloaders
    train_loader = torch.utils.data.DataLoader(dataset = train_data, batch_size = batch_size, shuffle = True)
    test_loader = torch.utils.data.DataLoader(dataset = test_data, batch_size = batch_size, shuffle = True)
    
    # Specify Factors
    lr = 0.0001 * (operations_dictionary[run_number_key])[1]
    num_epochs = 10
        
    # Single GPU
    if (operations_dictionary[run_number_key])[2] == 1:
        # Model Definition and Final Layer Edit
        net = models.resnet50()
        net.fc = nn.Linear(net.fc.in_features, 49)
        
        saved_model_path = ""
        
        if (operations_dictionary[run_number_key])[3]:
            # IF NEEDED TO IMPORT
            saved_model_path = '../SynchronizationProject/SaveData/Weights/' + str((operations_dictionary[run_number_key])[4]) + '.pth'
            net.load_state_dict(torch.load(saved_model_path))
        
        # Load Model Onto GPU
        device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        net.to(device)
        
        # Loss Function and Optimizer
        loss_function = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(net.parameters(), lr=lr)
        
        # Run Train
        train_history, val_history = train(net, loss_function, optimizer, train_loader, test_loader, num_epochs, device)
        
        # Save Model
        PATH = '../SynchronizationProject/SaveData/Weights/' + str(run_number_key) + '.pth'
        torch.save(net.state_dict(), PATH)
        
        new_row_data = {
                "Run": run_number_key,
                "BatchSize": batch_size,
                "LearningRate": lr,
                "TrainAccuracy": train_history["accuracy"][-1],
                "TrainLoss": train_history["loss"][-1],
                "TrainHistory": train_history,
                "ValAccuracy": val_history["accuracy"][-1],
                "ValLoss": val_history["loss"][-1],
                "ValHistory": val_history,
                "GPUCount": (operations_dictionary[run_number_key])[2],
                "GPUNumber": 1,
                "PreTrain": (operations_dictionary[run_number_key])[3],
                "SavePathInput": saved_model_path,
                "SavePathOutput": PATH
        }
            
        # Adding the new row to the DataFrame
        InfoSave.loc[len(InfoSave)] = new_row_data
    
    # Multiple GPU
    else:
        amount_of_GPUs_to_run = (operations_dictionary[run_number_key])[2]
        
        for run in range(amount_of_GPUs_to_run):
            # Define the start and end indices for the current run
            start_idx = (len(train_loader.dataset) // amount_of_GPUs_to_run) * run
            
            if run < (amount_of_GPUs_to_run - 1):
                end_idx = (len(train_loader.dataset) // amount_of_GPUs_to_run) * (run + 1)
            else:
                end_idx = len(train_loader.dataset)
            
            split_data_set = torch.utils.data.Subset(train_loader.dataset, range(start_idx, end_idx))
            split_train_loader = torch.utils.data.DataLoader(dataset=split_data_set, batch_size=batch_size, shuffle=True)
            
            # Model Definition and Final Layer Edit
            net = models.resnet50()
            net.fc = nn.Linear(net.fc.in_features, 49)
            
            saved_model_path = ""
            
            if (operations_dictionary[run_number_key])[3]:
                # IF NEEDED TO IMPORT
                saved_model_path = '../SynchronizationProject/SaveData/Weights/' + str((operations_dictionary[run_number_key])[4]) + '.pth'
                net.load_state_dict(torch.load(saved_model_path))
            
            # Load Model Onto GPU
            device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
            net.to(device)
            
            # Loss Function and Optimizer
            loss_function = nn.CrossEntropyLoss()
            optimizer = torch.optim.Adam(net.parameters(), lr=lr)
            
            #FIXTHIS TO WORK WITH MULTIPLE RUNS
            train_history, val_history = train(net, loss_function, optimizer, split_train_loader, test_loader, num_epochs, device)
            
            # Save Individual Weights
            PATH = '../SynchronizationProject/SaveData/Weights/' + str(run_number_key) + "T" + str(run) + '.pth'
            torch.save(net.state_dict(), PATH)
            
            new_row_data = {
                "Run": run_number_key,
                "BatchSize": batch_size,
                "LearningRate": lr,
                "TrainAccuracy": train_history["accuracy"][-1],
                "TrainLoss": train_history["loss"][-1],
                "TrainHistory": train_history,
                "ValAccuracy": val_history["accuracy"][-1],
                "ValLoss": val_history["loss"][-1],
                "ValHistory": val_history,
                "GPUCount": (operations_dictionary[run_number_key])[2],
                "GPUNumber": run + 1,
                "PreTrain": (operations_dictionary[run_number_key])[3],
                "SavePathInput": saved_model_path,
                "SavePathOutput": PATH
            }
            
            # Adding the new row to the DataFrame
            InfoSave.loc[len(InfoSave)] = new_row_data

file_path = '../SynchronizationProject/SaveData/DataFrames/InfoSave.csv'

InfoSave.to_csv(file_path, index=False)

0


KeyboardInterrupt: 

Evaluate The Model

In [6]:
file_path = '../SynchronizationProject/SaveData/DataFrames/InfoSave.csv'

InfoSave = pd.read_csv(file_path)

In [7]:
InfoSave

,Run,BatchSize,LearningRate,TrainAccuracy,TrainLoss,TrainHistory,ValAccuracy,ValLoss,ValHistory,GPUCount,GPUNumber,PreTrain,SavePathInput,SavePathOutput
0,0,32,0.0001,29.766119,2.530721,"{'loss': [3.4930564347435444, 3.41162627351050...",16.673579,3.380552,"{'loss': [3.5342165074850382, 3.59506788692976...",1,1,False,NaN,../SynchronizationProject/SaveData/Weights/0.pth
1,1,32,0.0001,22.628991,2.912521,"{'loss': [3.5299864712883444, 3.43312999500947...",9.373704,3.575293,"{'loss': [3.5124278695959794, 3.49547933904748...",3,1,False,NaN,../SynchronizationProject/SaveData/Weights/1T0...
2,1,32,0.0001,19.352325,3.014417,"{'loss': [3.5531147339764764, 3.45249549641328...",12.982165,3.580856,"{'loss': [3.520307826368432, 3.501597238214392...",3,2,False,NaN,../SynchronizationProject/SaveData/Weights/1T1...
3,1,32,0.0001,14.903829,3.129956,"{'loss': [3.557936040092917, 3.476790214987362...",12.567399,3.436109,"{'loss': [3.5215154917616593, 3.47128234411540...",3,3,False,NaN,../SynchronizationProject/SaveData/Weights/1T2...
4,2,64,0.0001,25.569260,2.713737,"{'loss': [3.4837466087192297, 3.39986238069832...",15.429282,3.255348,"{'loss': [3.4475874649850944, 3.41609198168704...",1,1,False,NaN,../SynchronizationProject/SaveData/Weights/2.pth
5,3,64,0.0001,23.005913,3.016223,"{'loss': [3.600028001345121, 3.422101405950693...",8.336790,3.701210,"{'loss': [3.5449120433706987, 3.51157576159427...",5,1,False,NaN,../SynchronizationProject/SaveData/Weights/3T0...
6,3,64,0.0001,25.058370,2.930626,"{'loss': [3.5564707059126635, 3.39519774913787...",10.991290,3.682562,"{'loss': [3.5440946942881535, 3.49703889144094...",5,2,False,NaN,../SynchronizationProject/SaveData/Weights/3T1...
7,3,64,0.0001,20.478625,2.953434,"{'loss': [3.598517115299518, 3.457311208431537...",7.341352,3.960901,"{'loss': [3.4908311680743567, 3.49983194627259...",5,3,False,NaN,../SynchronizationProject/SaveData/Weights/3T2...
8,3,64,0.0001,25.610257,2.911658,"{'loss': [3.601519465446472, 3.431241613167983...",8.212360,4.160569,"{'loss': [3.5688967642031217, 3.50579348363374...",5,4,False,NaN,../SynchronizationProject/SaveData/Weights/3T3...
9,3,64,0.0001,22.036984,3.032949,"{'loss': [3.5894748889482937, 3.40311352106241...",9.290751,3.795039,"{'loss': [3.5851886084205224, 3.53122147760893...",5,5,False,NaN,../SynchronizationProject/SaveData/Weights/3T4...


In [ ]:
# Plot Learning Curves
plot_learning_curve(train_history, val_history)

In [ ]:
# Put Model In Eval Mode
net.eval()

# Create Lists To Store The Predictions And True Labels
predictions = []
true_labels = []

# Iterate Through The Test Data
with torch.no_grad():
    for data in test_loader:

        # Load Data
        images, labels = data['image'].cuda(), data['cars'].cuda()
        outputs = net(images)

        true_labels.extend(labels.cpu().numpy())

        # Get Class With Highest Probability As Predicted Class
        _, predicted = torch.max(outputs, 1)

        # Convert Predictions To List And Append
        predictions.extend(predicted.cpu().numpy())

# Convert Predictions List And true_labels To NumPy Arrays
predictions = np.array(predictions)
true_labels = np.array(true_labels)

# Pull Class Names
class_names = (['AM'] + (pd.read_csv('../SynchronizationProject/stanford_cars_eec174/names_make.txt')['AM'].to_list()))

# Plot Everything
plot_confusion_matrix(true_labels, predictions, class_names)